# Setup - Ollama

In [1]:
# !pip install -qU gradio

In [ ]:
# !pip install colab-xterm
# %load_ext colabxterm

Lanch xtrem terminal in window.

> %xterm

Download and run ollama server

> curl -sSL https://ollama.ai/install.sh | sh && ollama serve

# Pre requisition
Validate if ollama runs

In [ ]:
! curl http://localhost:11434/api/pull -d '{  "model": "llama3.2" }'

In [6]:
!curl http://localhost:11434/api/generate -d '{  "model": "llama3.2",  "prompt":"Why is the sky blue?", "stream": false}'

{"model":"llama3.2","created_at":"2025-02-19T11:21:48.892546Z","response":"The sky appears blue because of a phenomenon called scattering, which occurs when sunlight interacts with the tiny molecules of gases in the Earth's atmosphere.\n\nHere's a simplified explanation:\n\n1. When sunlight enters the Earth's atmosphere, it encounters tiny molecules of gases such as nitrogen (N2) and oxygen (O2).\n2. These molecules are much smaller than the wavelength of light, which means they scatter the shorter (blue) wavelengths of light more than the longer (red) wavelengths.\n3. As a result, the blue light is dispersed in all directions by the tiny molecules, giving the sky its blue appearance.\n4. The scattering effect is more pronounced when the sun is overhead and the air is clear. This is why the sky often appears blue during the daytime.\n\nThere are a few other factors that can influence the color of the sky:\n\n* Atmospheric conditions: Dust, pollution, and water vapor in the air can scat

# Setup

In [10]:
import requests
import json
import gradio as gr

In [11]:
class CFG:
  model = "llama3.2"

# Funkcje

In [12]:
def generate(prompt, context, top_k, top_p, temp):
    r = requests.post('http://localhost:11434/api/generate',
                     json={
                         'model': CFG.model, 'prompt': prompt,
                         'context': context,
                         'options':{
                             'top_k': top_k,
                             'temperature':top_p,
                             'top_p': temp
                         } },
                     stream=False)
    r.raise_for_status()

    response = ""

    for line in r.iter_lines():
        body = json.loads(line)
        response_part = body.get('response', '')
        print(response_part)
        if 'error' in body:
            raise Exception(body['error'])

        response += response_part

        if body.get('done', False):
            context = body.get('context', [])
            return response, context

Ta funkcja `generate` służy do komunikacji z lokalnym serwerem API, który uruchamia model językowy. Przyjrzyjmy się jej działaniu krok po kroku:

Funkcja przyjmuje pięć parametrów:
- prompt: tekst, który zostanie przesłany do modelu
- context: kontekst dla modelu, pomocny w zachowaniu spójności rozmowy
- top_k, top_p i temp: parametry kontrolujące sposób generowania tekstu przez model

W pierwszej części funkcja wysyła zapytanie POST do serwera działającego na lokalnym komputerze (localhost) na porcie 11434. Zapytanie zawiera dane w formacie JSON z nazwą modelu (pobraną z klasy CFG), tekstem promptu, kontekstem oraz opcjami generowania.

Następnie funkcja sprawdza, czy zapytanie zakończyło się sukcesem (`raise_for_status()`). Jeśli wystąpił błąd HTTP, zostanie zgłoszony wyjątek.

W głównej części funkcji następuje przetwarzanie odpowiedzi z serwera. Odpowiedź jest przesyłana w formie strumienia linii tekstu, które funkcja przetwarza jedna po drugiej. Każda linia jest konwertowana z formatu JSON na obiekt Pythona.

Z każdej linii pobierany jest fragment odpowiedzi (response_part) i jest on wyświetlany na ekranie. Jeśli w odpowiedzi pojawi się błąd, funkcja zgłasza wyjątek z treścią błędu.

Fragmenty odpowiedzi są łączone w jeden tekst. Gdy serwer zasygnalizuje zakończenie generowania (pole 'done' ma wartość true), funkcja pobiera zaktualizowany kontekst i zwraca dwie wartości: pełną odpowiedź oraz nowy kontekst, który może być użyty w kolejnych wywołaniach.

Jest to przykład integracji z API modelu językowego, gdzie odpowiedź jest przesyłana strumieniowo (czyli część po części), co pozwala na wyświetlanie tekstu w czasie rzeczywistym, zamiast czekać na całą odpowiedź.

In [13]:
def chat(input, chat_history, top_k, top_p, temp):

    chat_history = chat_history or []

    global context
    output, context = generate(input, context, top_k, top_p, temp)

    chat_history.append((input, output))

    return chat_history, chat_history

Ta funkcja `chat` zarządza historią rozmowy między użytkownikiem a modelem językowym. Jej działanie jest bardzo interesujące z punktu widzenia zarządzania stanem konwersacji.

Przyjmuje ona pięć parametrów:
- input: wiadomość od użytkownika
- chat_history: lista zawierająca poprzednie wiadomości w rozmowie
- top_k, top_p, temp: parametry kontrolujące generowanie tekstu, przekazywane dalej do funkcji generate

Na początku funkcja sprawdza, czy chat_history istnieje - jeśli nie, tworzy pustą listę. Jest to zabezpieczenie przed sytuacją, gdy historia rozmowy nie została jeszcze zainicjowana.

Następnie funkcja używa słowa kluczowego `global`, aby uzyskać dostęp do zmiennej context zdefiniowanej poza funkcją. Jest to ważne, ponieważ context przechowuje informacje potrzebne do zachowania spójności rozmowy między kolejnymi wywołaniami.

W sercu funkcji znajduje się wywołanie poznane wcześniej funkcji generate. Przekazuje ona aktualną wiadomość użytkownika (input) wraz z kontekstem i parametrami generowania. Funkcja zwraca dwie wartości: wygenerowaną odpowiedź (output) oraz zaktualizowany kontekst, który zostaje zapisany w zmiennej globalnej.

Po otrzymaniu odpowiedzi od modelu, funkcja dodaje nową parę (wiadomość użytkownika, odpowiedź modelu) do historii rozmowy za pomocą metody append.

Na końcu funkcja zwraca dwie kopie zaktualizowanej historii rozmowy. Służą one dwóm różnym celom w interfejsie Gradio:
1. Pierwsza kopia aktualizuje widżet czatu, który wyświetla rozmowę użytkownikowi
2. Druga kopia aktualizuje wewnętrzny stan aplikacji, który przechowuje historię rozmowy między interakcjami

Jest to rozwiązanie, które pozwala na synchronizację tego, co widzi użytkownik, z tym, co "pamięta" aplikacja. Dzięki temu historia rozmowy jest zachowana nawet po odświeżeniu interfejsu lub ponownym uruchomieniu aplikacji.

# Chatbot

In [14]:
context = []

In [15]:
block = gr.Blocks()

with block:

    gr.Markdown("""<h1><center> My private chatbot </center></h1>
    """)

    message = gr.Textbox(placeholder="Type here")
    chatbot = gr.Chatbot()


    state = gr.State()

    with gr.Accordion("Advanced Settings", open=False):

      with gr.Row():
          top_k = gr.Slider(0.0,100.0, label="top_k", value=40, info="Reduces the probability of generating nonsense. A higher value (e.g. 100) will give more diverse answers, while a lower value (e.g. 10) will be more conservative. (Default: 40)")
          top_p = gr.Slider(0.0,1.0, label="top_p", value=0.9, info=" Works together with top-k. A higher value (e.g., 0.95) will lead to more diverse text, while a lower value (e.g., 0.5) will generate more focused and conservative text. (Default: 0.9)")
          temp = gr.Slider(0.0,2.0, label="temperature", value=0.8, info="The temperature of the model. Increasing the temperature will make the model answer more creatively. (Default: 0.8)")


    submit = gr.Button("SEND")

    submit.click(chat, inputs=[message, state, top_k, top_p, temp], outputs=[chatbot, state])

block.launch(debug=True, share = True)

/Users/marcinpilarczyk/projects/python/AI-Notes/_personal/course/gen_ai/day_3/.venv/lib/python3.10/site-packages/gradio/components/chatbot.py:284: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://462e0ef0be6d7795fa.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


I
'm
 just
 a
 language
 model
,
 so
 I
 don
't
 have
 emotions
 or
 feelings
 like
 humans
 do
.
 However
,
 I
'm
 functioning
 properly
 and
 ready
 to
 help
 with
 any
 questions
 or
 tasks
 you
 may
 have
!
 How
 about
 you
?
 How
's
 your
 day
 going
?

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://462e0ef0be6d7795fa.gradio.live


Ta część kodu tworzy interfejs graficzny aplikacji przy użyciu biblioteki Gradio. Przeanalizujmy jego strukturę i funkcjonalność:

Najpierw tworzymy blok - podstawowy kontener interfejsu Gradio - używając `gr.Blocks()`. Wszystkie elementy interfejsu będą umieszczone wewnątrz tego bloku.

W głównej części interfejsu znajduje się nagłówek HTML z tytułem "Muh private chatbot", wycentrowanym na stronie. Pod nim umieszczone są trzy kluczowe elementy:
- Pole tekstowe (gr.Textbox) z podpowiedzią "Type here", gdzie użytkownik wpisuje swoje wiadomości
- Komponent chatbota (gr.Chatbot), który wyświetla historię rozmowy
- Obiekt stanu (gr.State) przechowujący dane między interakcjami

Szczególnie interesujący jest rozwijaną sekcja "Advanced Settings". Zawiera ona trzy suwaki kontrolujące parametry generowania tekstu:

1. top_k (zakres 0-100, domyślnie 40): Pomaga redukować nonsensowne odpowiedzi. Wyższa wartość oznacza bardziej zróżnicowane odpowiedzi, niższa - bardziej zachowawcze.

2. top_p (zakres 0-1, domyślnie 0.9): Współdziała z top_k. Wyższa wartość prowadzi do bardziej różnorodnych odpowiedzi, niższa generuje bardziej skupiony i konserwatywny tekst.

3. temperature (zakres 0-2, domyślnie 0.8): Kontroluje "kreatywność" modelu. Wyższa temperatura skutkuje bardziej kreatywnymi odpowiedziami.

Na dole interfejsu znajduje się przycisk "SEND". Po jego kliknięciu wywoływana jest funkcja chat z odpowiednimi parametrami wejściowymi (wiadomość, stan, parametry generowania) i wyjściowymi (chatbot i stan).

Ostatnia linia uruchamia aplikację w trybie debugowania (debug=True) z opcją udostępniania (share=True), co pozwala na dostęp do interfejsu przez internet, nie tylko lokalnie.

To połączenie prostego w obsłudze interfejsu dla zwykłych użytkowników z zaawansowanymi opcjami dla bardziej doświadczonych osób, którzy chcą dostroić zachowanie modelu do swoich potrzeb.